In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sklearn, xgboost, joblib, sys, platform
print("sklearn:", sklearn.__version__)
print("xgboost:", xgboost.__version__)
print("joblib:", joblib.__version__)
print("python:", sys.version.split()[0], platform.platform())


sklearn: 1.6.1
xgboost: 3.0.5
joblib: 1.5.2
python: 3.12.11 Linux-6.6.97+-x86_64-with-glibc2.35


# Setup & preprocessing data

In [ ]:
import os, re, math, warnings
import numpy as np
import pandas as pd
import inspect

from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import math

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/duong/STLF/Data/merge_weather_energy_hanoi_20202025_cleaned.csv')
# df = pd.read_csv('/content/drive/MyDrive/STLF/Data/merge_weather_energy_hanoi_20202025_cleaned.csv')
df

,Temperature,Weather,Precipitation,Chance of snow,Humidity,Wind,Wind Gust,Wind Degree,Wind Direction,Cloud Cover,Visibility,timestamp,is_weekend,season,is_holiday,total_consumption_mw
0,25.4,Patchy rain possible,0.6,0.0,89.0,2.194444,4.388889,295.0,WNW,89.0,9.0,2020-01-01 00:00:00,0,winter,False,1790.10
1,25.1,Partly cloudy,0.0,0.0,90.0,2.611111,5.111111,297.0,WNW,34.0,10.0,2020-01-01 01:00:00,0,winter,False,1452.26
2,24.7,Patchy rain possible,0.0,0.0,91.0,2.805556,5.500000,309.0,NW,87.0,10.0,2020-01-01 02:00:00,0,winter,False,1483.75
3,24.5,Cloudy,0.0,0.0,92.0,2.611111,4.888889,325.0,NW,71.0,10.0,2020-01-01 03:00:00,0,winter,False,1890.07
4,24.1,Patchy rain possible,0.0,0.0,93.0,2.305556,4.000000,326.0,NNW,100.0,10.0,2020-01-01 04:00:00,0,winter,False,1371.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47444,26.8,Partly cloudy,0.0,0.0,84.0,1.888889,3.388889,109.0,ESE,56.0,10.0,2025-05-30 20:00:00,0,spring,False,2449.28
47445,26.5,Partly cloudy,0.0,0.0,86.0,2.388889,4.194444,126.0,SE,52.0,10.0,2025-05-30 21:00:00,0,spring,False,2554.05
47446,26.1,Patchy rain possible,0.0,0.0,88.0,2.388889,4.194444,149.0,SSE,84.0,10.0,2025-05-30 22:00:00,0,spring,False,1895.41
47447,25.6,Patchy rain possible,0.5,0.0,91.0,2.305556,4.111111,152.0,SSE,76.0,9.0,2025-05-30 23:00:00,0,spring,False,1558.67


In [ ]:
df['Chance of snow'].value_counts()

,count
Chance of snow,
0.0,47449


In [ ]:
CAT_COLS   = ['Weather','Wind Direction','season','is_holiday']
EXOG_NUM   = ['Temperature','Precipitation','Humidity',
              'Wind','Wind Gust','Wind Degree','Cloud Cover','Visibility']
HORIZON    = 24                       # forecast t+1…t+24
TARGET_LAGS   = range(1,49)           # lags of target (1…48)
ROLL_WINDOWS  = [3,6,12,24,48]        # rolling windows
EXOG_LAGS     = [0,1,3,6,12,24]       # lags for exogenous vars
TARGET_COL = 'total_consumption_mw'
TIME_COL   = 'timestamp'

In [ ]:
df[TIME_COL] = pd.to_datetime(df[TIME_COL])
df = df.sort_values(TIME_COL).reset_index(drop=True)

In [ ]:
# make sure categoricals are strings
for col in CAT_COLS:
    df[col] = df[col].astype(str)

In [ ]:
# ✓ 0) Sanity: is a GPU attached?
!nvidia-smi

# ✓ 1) Version check
import xgboost as xgb
print("XGBoost version:", xgb.__version__)

# ✓ 2) Minimal GPU training — if this runs, GPU is working
from sklearn.datasets import make_regression
from xgboost import XGBRegressor
import numpy as np

X, y = make_regression(n_samples=5000, n_features=50, noise=0.1, random_state=42)

xgb_gpu = XGBRegressor(
    tree_method="gpu_hist",       # <-- GPU trainer
    predictor="gpu_predictor",    # <-- GPU predictor
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)
xgb_gpu.fit(X, y)
print("Trained successfully with GPU!")


/bin/bash: line 1: nvidia-smi: command not found
XGBoost version: 3.0.5


XGBoostError: [01:46:37] /workspace/src/tree/updater_gpu_hist.cu:847: Check failed: ctx_->Ordinal() >= 0 (-1 vs. 0) : Must have at least one device
Stack trace:
  [bt] (0) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x2a6e7c) [0x7e40f40a6e7c]
  [bt] (1) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0xfd77b4) [0x7e40f4dd77b4]
  [bt] (2) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0xfdf506) [0x7e40f4ddf506]
  [bt] (3) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x63b5b2) [0x7e40f443b5b2]
  [bt] (4) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x63ca0d) [0x7e40f443ca0d]
  [bt] (5) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x68d33e) [0x7e40f448d33e]
  [bt] (6) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x77) [0x7e40f3fb6f57]
  [bt] (7) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7e413d832e2e]
  [bt] (8) /lib

# Feature engineering

In [ ]:
# add calendar/time features
df['hour']      = df[TIME_COL].dt.hour
df['dow']       = df[TIME_COL].dt.dayofweek
df['dom']       = df[TIME_COL].dt.day
df['month']     = df[TIME_COL].dt.month
df['doy']       = df[TIME_COL].dt.dayofyear
df['hour_sin']  = np.sin(2*np.pi*df['hour']/24)
df['hour_cos']  = np.cos(2*np.pi*df['hour']/24)
df['dow_sin']   = np.sin(2*np.pi*df['dow']/7)
df['dow_cos']   = np.cos(2*np.pi*df['dow']/7)
df['doy_sin']   = np.sin(2*np.pi*df['doy']/365.25)
df['doy_cos']   = np.cos(2*np.pi*df['doy']/365.25)

In [ ]:
# 2. feature engineering
# create lagged target features
for L in TARGET_LAGS:
    df[f'y_lag_{L}'] = df[TARGET_COL].shift(L)

# rolling statistics on the target, with one‑step shift to avoid leakage
for W in ROLL_WINDOWS:
    df[f'y_rollmean_{W}'] = df[TARGET_COL].shift(1).rolling(W, min_periods=1).mean()
    df[f'y_rollstd_{W}']  = df[TARGET_COL].shift(1).rolling(W, min_periods=1).std()

# lagged exogenous features
for col in EXOG_NUM:
    for L in EXOG_LAGS:
        df[f'{col}_lag{L}'] = df[col].shift(L)

# multi‑horizon labels
for h in range(1, HORIZON+1):
    df[f'y_t+{h}'] = df[TARGET_COL].shift(-h)

# drop rows with NaNs (from lagging & shifting)no
df_feat = df.dropna().reset_index(drop=True)
df_feat.shape

(47377, 157)

In [ ]:
np.max(df_feat['timestamp'])

Timestamp('2025-05-30 00:00:00')

In [ ]:
df_feat.shape[0] - 8*7*24

46033

In [ ]:
# 3. train/valid/test split by time
label_cols   = [f'y_t+{h}' for h in range(1, HORIZON+1)]
feature_cols = [c for c in df_feat.columns if c not in label_cols + [TIME_COL, TARGET_COL]]

X = df_feat[feature_cols]
Y = df_feat[label_cols].values

N        = len(X)
train_end= int(N - 2*8*7*24)
valid_end= int(N - 8*7*24)

X_train, Y_train = X.iloc[:train_end],    Y[:train_end]
X_valid, Y_valid = X.iloc[train_end:valid_end], Y[train_end:valid_end]
X_test,  Y_test  = X.iloc[valid_end:],    Y[valid_end:]

print(X_train.shape, Y_train.shape)
print(X_valid.shape, Y_valid.shape)
print(X_test.shape, Y_test.shape)

(44689, 131) (44689, 24)
(1344, 131) (1344, 24)
(1344, 131) (1344, 24)


In [12]:
# Optional (build future known ahead covariates)
DTYPE = np.float32
# Build Z for EXACT rows in df_feat, then split into train/val/test
def build_future_covariates(times, H, df, TIME_COL):
    df_key = df[[TIME_COL, 'is_holiday', 'season']].copy()
    df_key[TIME_COL] = pd.to_datetime(df_key[TIME_COL])
    df_key = df_key.set_index(TIME_COL).sort_index()

    Z_list = []
    for ts in times:
        row = []
        for h in range(1, H+1):
            t_h = pd.Timestamp(ts) + pd.Timedelta(hours=h)
            hour = t_h.hour; dow = t_h.dayofweek; doy = t_h.dayofyear
            v = [
                np.sin(2*np.pi*hour/24), np.cos(2*np.pi*hour/24),
                np.sin(2*np.pi*dow/7),  np.cos(2*np.pi*dow/7),
                np.sin(2*np.pi*doy/365.25), np.cos(2*np.pi*doy/365.25),
                1 if dow>=5 else 0,  # is_weekend
            ]
            if t_h in df_key.index:
                hol = str(df_key.loc[t_h, 'is_holiday'])
                sea = str(df_key.loc[t_h, 'season'])
            else:
                hol, sea = "False", None
            hol_vec = [int(hol=="False"), int(hol=="tet"), int(hol=="national")]
            sea_vec = [int(sea==s) for s in ["winter","spring","summer","autumn"]] if sea is not None else [0,0,0,0]
            v += hol_vec + sea_vec
            row.append(v)
        Z_list.append(row)
    return np.array(Z_list, dtype=DTYPE)  # (N, H, Ff)

# -- Build & split Z to align with X/Y splits defined above --
end_times_all = df_feat[TIME_COL].values
Z_all = build_future_covariates(end_times_all, HORIZON, df=df, TIME_COL=TIME_COL)

Z_train = Z_all[:train_end]
Z_val   = Z_all[train_end:valid_end]
Z_test  = Z_all[valid_end:]

print(Z_train.shape)
print(Z_val.shape)
print(Z_test.shape)

(44689, 24, 14)
(1344, 24, 14)
(1344, 24, 14)


In [ ]:
np.max(df_feat[:valid_end ]['timestamp'])

Timestamp('2025-04-04 00:00:00')

In [ ]:
# 4. preprocessor: one‑hot encode the 4 categorical columns, pass others through
preprocess = ColumnTransformer(
    [
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), CAT_COLS),
        ('num', 'passthrough', [c for c in feature_cols if c not in CAT_COLS])
    ],
    remainder='drop'
)


In [ ]:
# Scorer: average RMSE across 24 horizons (negated for maximization)
# =========================
def avg_neg_rmse(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    rmse_per_h = np.sqrt(np.mean((y_true - y_pred)**2, axis=0))
    return -float(np.mean(rmse_per_h))

In [ ]:
scorer = make_scorer(avg_neg_rmse, greater_is_better=True)
scorer

make_scorer(avg_neg_rmse, response_method='predict')

In [ ]:
# train+valid for tuning
X_trv = pd.concat([X_train, X_valid]); Y_trv = np.vstack([Y_train, Y_valid])

In [ ]:
tscv = TimeSeriesSplit(n_splits=3)

# XGBoost tuning

In [ ]:
# =========================
# 3) XGBoost tuning
# =========================
xgb_base = XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1, tree_method='gpu_hist')
xgb_pipe = Pipeline([('prep', preprocess),
                     ('model', MultiOutputRegressor(xgb_base, n_jobs=1))])

xgb_space = {
    'model__estimator__n_estimators':  [600, 800, 1000, 1200],
    'model__estimator__max_depth':     [6, 8, 10],
    'model__estimator__learning_rate': [0.03, 0.05, 0.08],
    'model__estimator__subsample':     [0.7, 0.8, 1.0],
    'model__estimator__colsample_bytree': [0.7, 0.8, 1.0],
    'model__estimator__min_child_weight': [1, 3, 5],
    'model__estimator__gamma':         [0, 0.1, 0.3],
    'model__estimator__reg_alpha':     [0.0, 0.1, 0.5],
    'model__estimator__reg_lambda':    [0.5, 1.0, 2.0],
}

xgb_tuner = RandomizedSearchCV(
    xgb_pipe, xgb_space, n_iter=12, cv=tscv, scoring=scorer,
    random_state=42, n_jobs=-1, verbose=1, refit=True
)
xgb_tuner.fit(X_trv, Y_trv)
print("Best XGB params:", xgb_tuner.best_params_)
print("Best XGB CV avg-RMSE:", -xgb_tuner.best_score_)

# evaluate on held-out test
Y_hat_xgb = xgb_tuner.predict(X_test)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [21:31:18] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [21:31:18] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [21:31:18] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserW

In [ ]:
# =========================
# 3) XGBoost tuning
# =========================

xgb_space = {
    'model__estimator__n_estimators':  [600, 800, 1000],
    'model__estimator__max_depth':     [6, 8],
    'model__estimator__learning_rate': [0.03, 0.05, 0.08],
    'model__estimator__subsample':     [0.7, 0.9],
    'model__estimator__colsample_bytree': [0.7, 0.9],
    'model__estimator__min_child_weight': [1, 3],
    'model__estimator__gamma':         [0, 0.1],
    'model__estimator__reg_alpha':     [0.0, 0.1],
    'model__estimator__reg_lambda':    [0.5, 1.0],
}

xgb_pipe = Pipeline([
    ('prep', preprocess),
    ('model', MultiOutputRegressor(XGBRegressor(objective='reg:squarederror',
                                                tree_method='hist', # 'gpu_hist' if GPU
                                                random_state=42, n_jobs=-1),
                                   n_jobs=1))
])

xgb_tuner = RandomizedSearchCV(
    xgb_pipe, xgb_space, n_iter=12, cv=tscv, scoring=scorer,
    random_state=42, n_jobs=-1, verbose=1, refit=True
)

xgb_tuner.fit(X_trv, Y_trv)
print("Best XGB params:", xgb_tuner.best_params_)
print("Best XGB CV avg-RMSE:", -xgb_tuner.best_score_)

# evaluate on held-out test
Y_hat_xgb = xgb_tuner.predict(X_test)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best XGB params: {'model__estimator__subsample': 0.7, 'model__estimator__reg_lambda': 0.5, 'model__estimator__reg_alpha': 0.0, 'model__estimator__n_estimators': 1000, 'model__estimator__min_child_weight': 3, 'model__estimator__max_depth': 6, 'model__estimator__learning_rate': 0.03, 'model__estimator__gamma': 0.1, 'model__estimator__colsample_bytree': 0.7}
Best XGB CV avg-RMSE: 320.7161990859629


In [ ]:
print("Best XGB params:", xgb_tuner.best_params_)
print("Best XGB CV avg-RMSE:", -xgb_tuner.best_score_)

Best XGB params: {'model__estimator__subsample': 0.7, 'model__estimator__reg_lambda': 0.5, 'model__estimator__reg_alpha': 0.0, 'model__estimator__n_estimators': 1000, 'model__estimator__min_child_weight': 3, 'model__estimator__max_depth': 6, 'model__estimator__learning_rate': 0.03, 'model__estimator__gamma': 0.1, 'model__estimator__colsample_bytree': 0.7}
Best XGB CV avg-RMSE: 320.7161990859629


In [ ]:
import joblib

# Save the lgb_tuner object
joblib.dump(xgb_tuner, 'xgb_tuner.pkl')

print("xgb_tuner saved to xgb_tuner.pkl")

xgb_tuner saved to xgb_tuner.pkl


In [ ]:
import joblib
xgb_tuner = joblib.load('/content/drive/MyDrive/duong/STLF/Saved_models/xgb_tuner_1.pkl')

In [ ]:
xgb_tuner.best_params_

{'model__estimator__subsample': 0.7,
 'model__estimator__reg_lambda': 0.5,
 'model__estimator__reg_alpha': 0.0,
 'model__estimator__n_estimators': 1000,
 'model__estimator__min_child_weight': 3,
 'model__estimator__max_depth': 6,
 'model__estimator__learning_rate': 0.03,
 'model__estimator__gamma': 0.1,
 'model__estimator__colsample_bytree': 0.7}

In [ ]:
# 70-15-15 split
import xgboost as xgb
# 1) Extract best params from your tuner and map to xgb.train names
bp = xgb_tuner.best_params_
def g(k, default=None): return bp.get(f"model__estimator__{k}", default)

xgb_params = {
    "max_depth":         g("max_depth"),
    "eta":               g("learning_rate"),
    "subsample":         g("subsample"),
    "colsample_bytree":  g("colsample_bytree"),
    "min_child_weight":  g("min_child_weight"),
    "gamma":             g("gamma"),
    "lambda":            g("reg_lambda"),
    "alpha":             g("reg_alpha"),
    "objective":         "reg:squarederror",
    "eval_metric":       "rmse",
    "verbosity":         0,
    #"tree_method":       "gpu_hist",   # set to 'gpu_hist' if you have a GPU
}
num_boost_round = g("n_estimators", 4000)

# 2) Fit encoder on train+valid once; transform arrays for fast training
X_trv_enc  = preprocess.fit_transform(X_trv)
X_test_enc = preprocess.transform(X_test)

# 3) Early-stopping split: last 15% of TRV as eval (keeps time order)
cut = int(0.85 * X_trv_enc.shape[0])
X_tr, X_ev = X_trv_enc[:cut], X_trv_enc[cut:]
Y_tr, Y_ev = Y_trv[:cut],     Y_trv[cut:]

# 4) Train 24 boosters with early stopping
models_xgb = []
for h in range(HORIZON):  # 0..23 for t+1..t+24
    dtrain = xgb.DMatrix(X_tr, label=Y_tr[:, h])
    dvalid = xgb.DMatrix(X_ev, label=Y_ev[:, h])
    booster = xgb.train(
        params=xgb_params,
        dtrain=dtrain,
        num_boost_round=num_boost_round,
        evals=[(dvalid, "valid")],
        early_stopping_rounds=100,
        verbose_eval=False,
    )
    models_xgb.append(booster)

# 5) Predict on test and report metrics
dtest = xgb.DMatrix(X_test_enc)
Y_hat_xgb = np.column_stack([
    m.predict(dtest, iteration_range=(0, m.best_iteration + 1)) for m in models_xgb
])

def safe_mape(y_true, y_pred, eps=1e-6):
    y_true = np.asarray(y_true)
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), eps, None))) * 100.0

rmse_h_xgb = [math.sqrt(mean_squared_error(Y_test[:, h], Y_hat_xgb[:, h])) for h in range(HORIZON)]
nrmse_h_xgb = [rm / (np.mean(Y_test[:, h]) + 1e-6) * 100 for h, rm in enumerate(rmse_h_xgb)]
mape_h_xgb  = [safe_mape(Y_test[:, h], Y_hat_xgb[:, h]) for h in range(HORIZON)]

print("Per-h RMSE:", [round(x,2) for x in rmse_h_xgb])
print("Avg RMSE:  {:.3f}".format(np.mean(rmse_h_xgb)))
print("Avg nRMSE: {:.2f}%".format(np.mean(nrmse_h_xgb)))
print("Avg MAPE:  {:.2f}%".format(np.mean(mape_h_xgb)))

NameError: name 'xgb_tuner' is not defined

### Without Z

In [ ]:
# 8lastweeks split
import xgboost as xgb
# Extract best params from your tuner and map to xgb.train names
bp = xgb_tuner.best_params_
def g(k, default=None): return bp.get(f"model__estimator__{k}", default)

xgb_params = {
    "max_depth":         g("max_depth"),
    "eta":               g("learning_rate"),
    "subsample":         g("subsample"),
    "colsample_bytree":  g("colsample_bytree"),
    "min_child_weight":  g("min_child_weight"),
    "gamma":             g("gamma"),
    "lambda":            g("reg_lambda"),
    "alpha":             g("reg_alpha"),
    "objective":         "reg:squarederror",
    "eval_metric":       "rmse",
    "verbosity":         0,
    "tree_method":       "gpu_hist",   # set to 'gpu_hist' if you have a GPU
}
num_boost_round = g("n_estimators", 4000)

X_train_enc  = preprocess.fit_transform(X_train)
X_valid_enc = preprocess.transform(X_valid)
X_test_enc = preprocess.transform(X_test)

early_stopping_rounds = 100

# Train 24 boosters with early stopping
models_xgb = []
for h in range(HORIZON):  # 0..23 for t+1..t+24
    dtrain = xgb.DMatrix(X_train_enc, label=Y_train[:, h])
    dvalid = xgb.DMatrix(X_valid_enc, label=Y_valid[:, h])
    booster = xgb.train(
        params=xgb_params,
        dtrain=dtrain,
        num_boost_round=num_boost_round,
        evals=[(dvalid, "valid")],
        early_stopping_rounds=early_stopping_rounds,
        verbose_eval=False,
    )
    models_xgb.append(booster)

# Predict on test and report metrics
dtest = xgb.DMatrix(X_test_enc)
Y_hat_xgb = np.column_stack([
    m.predict(dtest, iteration_range=(0, m.best_iteration + 1)) for m in models_xgb
])

def safe_mape(y_true, y_pred, eps=1e-6):
    y_true = np.asarray(y_true)
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), eps, None))) * 100.0

mae_h_xgb = np.mean(np.abs(Y_test - Y_hat_xgb), axis=0)
rmse_h_xgb = [math.sqrt(mean_squared_error(Y_test[:, h], Y_hat_xgb[:, h])) for h in range(HORIZON)]
nrmse_h_xgb = [rm / (np.mean(Y_test[:, h]) + 1e-6) * 100 for h, rm in enumerate(rmse_h_xgb)]
mape_h_xgb  = [safe_mape(Y_test[:, h], Y_hat_xgb[:, h]) for h in range(HORIZON)]

print("Per-h RMSE:", [round(x,2) for x in rmse_h_xgb])
print("Avg RMSE:  {:.3f}".format(np.mean(rmse_h_xgb)))
print("Avg nRMSE: {:.2f}%".format(np.mean(nrmse_h_xgb)))
print("Avg MAE:  {:.3f}".format(np.mean(mae_h_xgb)))
print("Avg MAPE:  {:.2f}%".format(np.mean(mape_h_xgb)))

Per-h RMSE: [286.11, 295.73, 272.58, 280.61, 275.54, 271.19, 273.26, 263.52, 273.82, 286.92, 283.05, 296.13, 292.93, 312.29, 307.13, 304.76, 299.94, 282.78, 287.76, 306.48, 300.22, 301.2, 319.0, 304.15]
Avg RMSE:  290.712
Avg nRMSE: 11.75%
Avg MAPE:  8.41%


## With future known ahead Z

In [ ]:
# =========================
# XGBoost + known-ahead Z
# Train on TRAIN only; predict on VALID and TEST
# =========================
DTYPE = np.float32

# Fit preprocessor on TRAIN only; transform VALID & TEST
X_train_enc = preprocess.fit_transform(X_train)
X_val_enc   = preprocess.transform(X_valid)
X_test_enc  = preprocess.transform(X_test)

print(X_train_enc.shape)
print(X_val_enc.shape)
print(X_test_enc.shape)

# 3) Map tuned params to xgb.train
bp = xgb_tuner.best_params_  # already loaded from your joblib
def g(k, default=None): return bp.get(f"model__estimator__{k}", default)

xgb_params = {
    "max_depth":         g("max_depth"),
    "eta":               g("learning_rate"),
    "subsample":         g("subsample"),
    "colsample_bytree":  g("colsample_bytree"),
    "min_child_weight":  g("min_child_weight"),
    "gamma":             g("gamma"),
    "lambda":            g("reg_lambda"),
    "alpha":             g("reg_alpha"),
    "objective":         "reg:squarederror",
    "eval_metric":       "rmse",
    "verbosity":         0,
    "tree_method":       "gpu_hist",   # switch to 'hist' if no GPU
}
num_boost_round = g("n_estimators", 1500)
early_stopping_rounds = 100

# 4) Train per-horizon on TRAIN, early-stop on VALID; predict both VALID and TEST
models_xgb = []
y_hat_val_list  = []
y_hat_test_list = []

for h in range(HORIZON):  # 0..23 => t+1..t+24
    Xtr_h = np.hstack([X_train_enc, Z_train[:, h, :]]).astype(DTYPE)
    Xva_h = np.hstack([X_val_enc,   Z_val[:,   h, :]]).astype(DTYPE)
    Xte_h = np.hstack([X_test_enc,  Z_test[:,  h, :]]).astype(DTYPE)

    dtrain = xgb.DMatrix(Xtr_h, label=Y_train[:, h])
    dvalid = xgb.DMatrix(Xva_h, label=Y_valid[:, h])

    booster = xgb.train(
        params=xgb_params,
        dtrain=dtrain,
        num_boost_round=num_boost_round,
        evals=[(dvalid, "valid")],
        early_stopping_rounds=early_stopping_rounds,
        verbose_eval=False,
    )
    models_xgb.append(booster)

    # predictions for VAL & TEST under the SAME horizon-specific features
    y_hat_val_list.append(
        booster.predict(xgb.DMatrix(Xva_h), iteration_range=(0, booster.best_iteration + 1))
    )
    y_hat_test_list.append(
        booster.predict(xgb.DMatrix(Xte_h), iteration_range=(0, booster.best_iteration + 1))
    )

Y_hat_val  = np.column_stack(y_hat_val_list)   # shape (N_val,  H)
Y_hat_test = np.column_stack(y_hat_test_list)  # shape (N_test, H)

# 5) Optional: metrics for quick inspection
def safe_mape(y_true, y_pred, eps=1e-6):
    y_true = np.asarray(y_true)
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), eps, None))) * 100.0

mae_h_xgb_z = np.mean(np.abs(Y_test - Y_hat_test), axis=0)
rmse_h_xgb_z = [math.sqrt(mean_squared_error(Y_test[:, h], Y_hat_test[:, h])) for h in range(HORIZON)]
nrmse_h_xgb_z = [rm / (np.mean(Y_test[:, h]) + 1e-6) * 100 for h, rm in enumerate(rmse_h_xgb_z)]
mape_h_xgb_z  = [safe_mape(Y_test[:, h], Y_hat_test[:, h]) for h in range(HORIZON)]

print("Per-h RMSE:", [round(x,2) for x in rmse_h_xgb_z])
print("Avg RMSE:  {:.3f}".format(np.mean(rmse_h_xgb_z)))
print("Avg nRMSE: {:.2f}%".format(np.mean(nrmse_h_xgb_z)))
print("Avg MAE:  {:.3f}".format(np.mean(mae_h_xgb_z)))
print("Avg MAPE:  {:.2f}%".format(np.mean(mape_h_xgb_z)))


(44689, 155)
(1344, 155)
(1344, 155)
Per-h RMSE: [284.64, 289.16, 264.31, 271.62, 269.46, 274.13, 268.79, 263.21, 269.48, 286.15, 293.18, 295.98, 269.63, 302.86, 284.66, 295.39, 295.11, 276.92, 280.03, 304.86, 296.26, 292.53, 318.07, 300.43]
Avg RMSE:  285.286
Avg nRMSE: 11.53%
Avg MAPE:  8.13%


# LightGBM tuning

In [ ]:
import numpy as np, lightgbm as lgb
X = np.random.rand(1000, 100); y = np.random.rand(1000)
d = lgb.Dataset(X, label=y)
lgb.train({"objective":"regression","device_type":"gpu"}, d, num_boost_round=1)
print("GPU works")

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 100
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 100 dense feature groups (0.10 MB) transferred to GPU in 0.001087 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.498480
GPU works


In [ ]:
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV

# Base estimator
lgb_base = LGBMRegressor(
    objective="regression",
    random_state=42,
    n_jobs=-1,          # let the trees parallelize
    boosting_type="gbdt",
    verbose=-1,
    max_bin=127,
    #device_type="gpu"
)

# Pipeline (avoid double over-parallelization)
lgb_pipe = Pipeline([
    ("prep", preprocess),
    ("model", MultiOutputRegressor(lgb_base, n_jobs=1))  # keep this at 1 if RandomizedSearchCV uses n_jobs=-1
])

# Search space (lean but effective)
lgb_space = {
  "model__estimator__n_estimators": [300, 500, 700],
  "model__estimator__learning_rate":[0.05, 0.08],
  "model__estimator__num_leaves":  [63, 127],
  "model__estimator__max_depth":   [-1, 8],
  "model__estimator__subsample":   [0.7, 0.9],
  "model__estimator__colsample_bytree":[0.7, 0.9],
  "model__estimator__min_child_samples":[20, 40],
  "model__estimator__reg_alpha":   [0.0, 0.1],
  "model__estimator__reg_lambda":  [0.1, 1.0],
}

# Randomized search (fast pass; bump n_iter if you want wider exploration)
lgb_tuner = RandomizedSearchCV(
    lgb_pipe,
    lgb_space,
    n_iter=8,
    cv=tscv,            # your TimeSeriesSplit
    scoring=scorer,     # your avg-neg-RMSE scorer
    random_state=42,
    n_jobs=-1,          # parallelize candidates × folds
    verbose=1,
    refit=True
)

lgb_tuner.fit(X_trv, Y_trv)

print("Best LGB params:", lgb_tuner.best_params_)
print("Best LGB CV avg-RMSE:", -lgb_tuner.best_score_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


KeyboardInterrupt: 

In [ ]:
# ===== LGB final refit with early stopping (continue from your state) =====
import numpy as np, math, lightgbm as lgb
from sklearn.metrics import mean_squared_error

# 1) map best params from tuner to lgb.train
bp = lgb_tuner.best_params_
g  = lambda k, d=None: bp.get(f"model__estimator__{k}", d)

lgb_params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": g("learning_rate", 0.05),
    "num_leaves": g("num_leaves", 127),
    "max_depth": g("max_depth", -1),
    "feature_fraction": g("colsample_bytree", 0.8),   # alias
    "bagging_fraction": g("subsample", 0.8),          # alias
    "bagging_freq": bp.get("model__estimator__subsample_freq", 0),
    "min_data_in_leaf": g("min_child_samples", 20),
    "lambda_l1": g("reg_alpha", 0.0),
    "lambda_l2": g("reg_lambda", 0.1),
    "verbosity": -1,
    "num_threads": -1,
    # Optional GPU:
    "device_type": "gpu",
    # Optional speed knob during refit:
    "max_bin": 255,
}
# let ES find the best iteration (set bigger than tuned n_estimators)
num_boost_round = 4000
early_stopping_rounds = 100

# 2) encode once
X_trv_enc  = preprocess.fit_transform(X_trv)
X_test_enc = preprocess.transform(X_test)

# 3) carve last 15% of TRV as eval set (keep chronological order)
cut = int(0.85 * X_trv_enc.shape[0])
X_tr, X_ev = X_trv_enc[:cut], X_trv_enc[cut:]
Y_tr, Y_ev = Y_trv[:cut],     Y_trv[cut:]

# 4) train 24 boosters with ES
models_lgb = []
for h in range(HORIZON):
    dtr = lgb.Dataset(X_tr, label=Y_tr[:, h])
    dev = lgb.Dataset(X_ev, label=Y_ev[:, h], reference=dtr)
    booster = lgb.train(
        params=lgb_params,
        train_set=dtr,
        num_boost_round=num_boost_round,
        valid_sets=[dev],
        callbacks=[lgb.early_stopping(early_stopping_rounds), lgb.log_evaluation(-1)]
    )
    models_lgb.append(booster)

# 5) predict & metrics
Y_hat_lgb = np.column_stack([m.predict(X_test_enc, num_iteration=m.best_iteration) for m in models_lgb])

def safe_mape(y_true, y_pred, eps=1e-6):
    y_true = np.asarray(y_true)
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), eps, None))) * 100.0

rmse_h_lgb = [math.sqrt(mean_squared_error(Y_test[:, h], Y_hat_lgb[:, h])) for h in range(HORIZON)]
nrmse_h_lgb = [rm / (np.mean(Y_test[:, h]) + 1e-6) * 100 for h, rm in enumerate(rmse_h_lgb)]
mape_h_lgb  = [safe_mape(Y_test[:, h], Y_hat_lgb[:, h]) for h in range(HORIZON)]

print("Per-horizon RMSE:", [round(x,2) for x in rmse_h_lgb])
print("Avg RMSE:  {:.3f}".format(np.mean(rmse_h_lgb)))
print("Avg nRMSE: {:.2f}%".format(np.mean(nrmse_h_lgb)))
print("Avg MAPE:  {:.2f}%".format(np.mean(mape_h_lgb)))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[497]	valid_0's rmse: 262.679
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[549]	valid_0's rmse: 264.763
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[578]	valid_0's rmse: 271.19
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[520]	valid_0's rmse: 273.567
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[805]	valid_0's rmse: 282.022
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[930]	valid_0's rmse: 291.961
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[697]	valid_0's rmse: 299.357
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[916]	valid_0's

In [ ]:
import os
BASE_DIR = "/content/drive/MyDrive/duong/STLF/Saved_models/lgbm_newsplit" # luongphuc.lpu
# BASE_DIR = "/content/drive/MyDrive/STLF/Saved_models/lgbm_newsplit" # ducan
os.makedirs(BASE_DIR, exist_ok=True)

In [ ]:
import os, json, joblib, numpy as np, gc
from sklearn.model_selection import ParameterSampler
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor

In [ ]:
# ==== config for Drive paths ====
N_ITER = 8  # same as your RandomizedSearchCV
PROGRESS_JSON = os.path.join(BASE_DIR, "lgbm_tune_progress.json")
PARAMS_JSON   = os.path.join(BASE_DIR, "lgbm_param_list.json")  # to keep the same sequence on resume
TRIAL_CKP_FMT = os.path.join(BASE_DIR, "lgbm_checkpoint_trial_{:03d}.pkl")
BEST_MODEL_PKL= os.path.join(BASE_DIR, "lgbm_best_model.pkl")

In [ ]:
# Base estimator
lgb_base = LGBMRegressor(
    objective="regression",
    random_state=42,
    n_jobs=-1,          # let the trees parallelize
    boosting_type="gbdt",
    verbose=-1,
    max_bin=127,
    #device_type="gpu"
)

# Pipeline (avoid double over-parallelization)
lgb_pipe = Pipeline([
    ("prep", preprocess),
    ("model", MultiOutputRegressor(lgb_base, n_jobs=1))  # keep this at 1 if RandomizedSearchCV uses n_jobs=-1
])

# Search space (lean but effective)
lgb_space = {
  "model__estimator__n_estimators": [300, 500, 700],
  "model__estimator__learning_rate":[0.05, 0.08],
  "model__estimator__num_leaves":  [63, 127],
  "model__estimator__max_depth":   [-1, 8],
  "model__estimator__subsample":   [0.7, 0.9],
  "model__estimator__colsample_bytree":[0.7, 0.9],
  "model__estimator__min_child_samples":[20, 40],
  "model__estimator__reg_alpha":   [0.0, 0.1],
  "model__estimator__reg_lambda":  [0.1, 1.0],
}

# ==== build / load param list deterministically ====
if os.path.exists(PARAMS_JSON):
    with open(PARAMS_JSON, "r") as f:
        param_list = json.load(f)
else:
    from sklearn.model_selection import ParameterSampler
    param_list = list(ParameterSampler(lgb_space, n_iter=N_ITER, random_state=42))
    with open(PARAMS_JSON, "w") as f:
        json.dump(param_list, f)

# ==== load progress if exists ====
if os.path.exists(PROGRESS_JSON):
    with open(PROGRESS_JSON, "r") as f:
        prog = json.load(f)
    completed = set(prog.get("completed_indices", []))
    best_score = prog.get("best_score", -np.inf)
    best_params = prog.get("best_params", None)
else:
    completed, best_score, best_params = set(), -np.inf, None
    prog = {"completed_indices": [], "best_score": best_score, "best_params": best_params}

print(f"Resuming: {len(completed)}/{len(param_list)} trials completed.")
print("Checkpoints folder:", BASE_DIR)

# ==== helper: evaluate one param set with your tscv + scorer ====
def cv_score_for_params(params):
    lgb_base = LGBMRegressor(
        objective="regression",
        random_state=42,
        n_jobs=-1,
        boosting_type="gbdt",
        verbose=-1,
        max_bin=127,
        # GPU if available, else comment out:
        #device_type="gpu"
    )
    pipe = Pipeline([
        ("prep", preprocess),
        ("model", MultiOutputRegressor(lgb_base, n_jobs=1))
    ])
    pipe.set_params(**params)

    fold_scores = []
    for fold_idx, (tr_idx, va_idx) in enumerate(tscv.split(X_trv)):
        X_tr, X_va = X_trv.iloc[tr_idx], X_trv.iloc[va_idx]
        Y_tr, Y_va = Y_trv[tr_idx],      Y_trv[va_idx]

        pipe_fold = clone(pipe)
        pipe_fold.fit(X_tr, Y_tr)
        s = scorer(pipe_fold, X_va, Y_va)   # your avg-neg-RMSE scorer
        fold_scores.append(s)
        # free memory between folds
        del pipe_fold; gc.collect()
    return float(np.mean(fold_scores)), pipe  # return mean score and last-fitted pipe

# ==== main loop with Drive checkpoints ====
for i, params in enumerate(param_list):
    if i in completed:
        print(f"Skipping trial {i+1}: already completed.")
        continue

    print(f"\n--- Trial {i+1}/{len(param_list)} ---")
    print("Params:", params)
    try:
        score, fitted_pipe = cv_score_for_params(params)
        print("CV score (avg-neg-RMSE):", score)

        # save trial checkpoint to Drive
        joblib.dump(fitted_pipe, TRIAL_CKP_FMT.format(i+1), compress=3)

        # update best
        if score > best_score:
            best_score = score
            best_params = params
            joblib.dump(fitted_pipe, BEST_MODEL_PKL, compress=3)
            print(">> New best! Saved:", BEST_MODEL_PKL)

        # update progress JSON on Drive
        completed.add(i)
        prog["completed_indices"] = sorted(list(completed))
        prog["best_score"] = best_score
        prog["best_params"] = best_params
        with open(PROGRESS_JSON, "w") as f:
            json.dump(prog, f)
        print("Progress saved to:", PROGRESS_JSON)

        # free memory
        del fitted_pipe; gc.collect()

    except Exception as e:
        print(f"Trial {i+1} failed: {e}")
        # still save progress before stopping/continuing
        prog["completed_indices"] = sorted(list(completed))
        prog["best_score"] = best_score
        prog["best_params"] = best_params
        with open(PROGRESS_JSON, "w") as f:
            json.dump(prog, f)
        break  # or 'continue' if you want to try the next candidate

# summary
if best_params is not None:
    print("\nBest params so far:", best_params)
    print("Best CV avg-RMSE:", -best_score)
    # Optional: reload best model later
    # best_model = joblib.load(BEST_MODEL_PKL)
else:
    print("No successful trials yet.")

print("Best LGB params:", lgb_tuner.best_params_)
print("Best LGB CV avg-RMSE:", -lgb_tuner.best_score_)

Resuming: 0/8 trials completed.
Checkpoints folder: /content/drive/MyDrive/STLF/Saved_models/lgbm_newsplit

--- Trial 1/8 ---
Params: {'model__estimator__subsample': 0.7, 'model__estimator__reg_lambda': 0.1, 'model__estimator__reg_alpha': 0.1, 'model__estimator__num_leaves': 127, 'model__estimator__n_estimators': 700, 'model__estimator__min_child_samples': 40, 'model__estimator__max_depth': -1, 'model__estimator__learning_rate': 0.08, 'model__estimator__colsample_bytree': 0.9}
CV score (avg-neg-RMSE): -336.73368487022674
>> New best! Saved: /content/drive/MyDrive/STLF/Saved_models/lgbm_newsplit/lgbm_best_model.pkl
Progress saved to: /content/drive/MyDrive/STLF/Saved_models/lgbm_newsplit/lgbm_tune_progress.json

--- Trial 2/8 ---
Params: {'model__estimator__subsample': 0.7, 'model__estimator__reg_lambda': 0.1, 'model__estimator__reg_alpha': 0.1, 'model__estimator__num_leaves': 63, 'model__estimator__n_estimators': 700, 'model__estimator__min_child_samples': 20, 'model__estimator__max_de

NameError: name 'lgb_tuner' is not defined

In [ ]:
lgb_tuner = joblib.load(BEST_MODEL_PKL)
lgb_tuner

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Weather', 'Wind Direction',
                                                   'season', 'is_holiday']),
                                                 ('num', 'passthrough',
                                                  ['Temperature',
                                                   'Precipitation',
                                                   'Chance of snow', 'Humidity',
                                                   'Wind', 'Wind Gust',
                                                   'Wind Degree', 'Cloud Cover',
                                                   'Visibility', 'is_weekend',
                                                   'hour', 'dow', 'dom',
                                                   'month', 'd...
                                                   'doy_cos', 'y_lag_1',
                                                   'y_lag_2', 'y_lag_3',
                                                   'y_lag_4', 'y_lag_5',
                                                   'y_lag_6', 'y_lag_7',
                                                   'y_lag_8', 'y_lag_9', ...])])),
                ('model',
                 MultiOutputRegressor(estimator=LGBMRegressor(colsample_bytree=0.7,
                                                              learning_rate=0.05,
                                                              max_bin=127,
                                                              max_depth=8,
                                                              n_estimators=300,
                                                              n_jobs=-1,
                                                              num_leaves=63,
                                                              objective='regression',
                                                              random_state=42,
                                                              reg_lambda=0.1,
                                                              subsample=0.9,
                                                              verbose=-1),
                                      n_jobs=1))])

In [ ]:
with open(os.path.join(BASE_DIR, PROGRESS_JSON), "r") as f:
    prog = json.load(f)
best_params_lgb = prog.get("best_params", None)
best_params_lgb

{'model__estimator__subsample': 0.9,
 'model__estimator__reg_lambda': 0.1,
 'model__estimator__reg_alpha': 0.0,
 'model__estimator__num_leaves': 63,
 'model__estimator__n_estimators': 300,
 'model__estimator__min_child_samples': 20,
 'model__estimator__max_depth': 8,
 'model__estimator__learning_rate': 0.05,
 'model__estimator__colsample_bytree': 0.7}

In [ ]:
# Save best params dictionary to Drive
with open(os.path.join(BASE_DIR, "lgbm_best_params.json"), "w") as f:
    json.dump(best_params, f)
print(">> Saved best params to:", os.path.join(BASE_DIR, "lgbm_best_params.json"))

>> Saved best params to: /content/drive/MyDrive/STLF/Saved_models/lgbm_newsplit/lgbm_best_params.json


### Without Z

In [ ]:
# ===== LGB final refit with early stopping (continue from your state) =====
import numpy as np, math, lightgbm as lgb
from sklearn.metrics import mean_squared_error

# 1) map best params from tuner to lgb.train
bp = best_params_lgb
g  = lambda k, d=None: bp.get(f"model__estimator__{k}", d)

lgb_params = {
    "seed": 42,                       # master seed (alias: random_state)
    "bagging_seed": 42,
    "feature_fraction_seed": 42,
    "data_random_seed": 42,
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": g("learning_rate", 0.05),
    "num_leaves": g("num_leaves", 127),
    "max_depth": g("max_depth", -1),
    "feature_fraction": g("colsample_bytree", 0.8),   # alias
    "bagging_fraction": g("subsample", 0.8),          # alias
    "bagging_freq": bp.get("model__estimator__subsample_freq", 0),
    "min_data_in_leaf": g("min_child_samples", 20),
    "lambda_l1": g("reg_alpha", 0.0),
    "lambda_l2": g("reg_lambda", 0.1),
    "verbosity": -1,
    "num_threads": -1,
    # Optional GPU:
    "device_type": "gpu",
    # Optional speed knob during refit:
    "max_bin": 255,
}
# let ES find the best iteration (set bigger than tuned n_estimators)
num_boost_round = g("n_estimators", 4000)
early_stopping_rounds = 100

# 2) encode once
X_train_enc  = preprocess.fit_transform(X_train)
X_valid_enc = preprocess.transform(X_valid)
X_test_enc = preprocess.transform(X_test)

# train 24 boosters with ES
models_lgb = []
for h in range(HORIZON):
    dtr = lgb.Dataset(X_train_enc, label=Y_train[:, h])
    dev = lgb.Dataset(X_valid_enc, label=Y_valid[:, h], reference=dtr)
    booster = lgb.train(
        params=lgb_params,
        train_set=dtr,
        num_boost_round=num_boost_round,
        valid_sets=[dev],
        callbacks=[lgb.early_stopping(early_stopping_rounds), lgb.log_evaluation(-1)]
    )
    models_lgb.append(booster)

# 5) predict & metrics
Y_hat_lgb = np.column_stack([m.predict(X_test_enc, num_iteration=m.best_iteration) for m in models_lgb])

def safe_mape(y_true, y_pred, eps=1e-6):
    y_true = np.asarray(y_true)
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), eps, None))) * 100.0

mae_h_lgb = np.mean(np.abs(Y_test - Y_hat_lgb), axis=0)
rmse_h_lgb = [math.sqrt(mean_squared_error(Y_test[:, h], Y_hat_lgb[:, h])) for h in range(HORIZON)]
nrmse_h_lgb = [rm / (np.mean(Y_test[:, h]) + 1e-6) * 100 for h, rm in enumerate(rmse_h_lgb)]
mape_h_lgb  = [safe_mape(Y_test[:, h], Y_hat_lgb[:, h]) for h in range(HORIZON)]

print("Per-horizon RMSE:", [round(x,2) for x in rmse_h_lgb])
print("Avg RMSE:  {:.3f}".format(np.mean(rmse_h_lgb)))
print("Avg nRMSE: {:.2f}%".format(np.mean(nrmse_h_lgb)))
print("Avg MAE:  {:.3f}".format(np.mean(mae_h_lgb)))
print("Avg MAPE:  {:.2f}%".format(np.mean(mape_h_lgb)))


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[291]	valid_0's rmse: 223.891
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[296]	valid_0's rmse: 229.095
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[300]	valid_0's rmse: 211.527
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[292]	valid_0's rmse: 212.14
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[299]	valid_0's rmse: 210.652
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[291]	valid_0's rmse: 206.182
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[299]	valid_0's rmse: 209.434
Training until valida

### With future known ahead Z

In [ ]:
# =========================
# LightGBM + known-ahead Z
# Train on TRAIN only; predict on VALID and TEST
# =========================
DTYPE = np.float32

# Fit preprocessor on TRAIN only; transform VALID & TEST
X_train_enc = preprocess.fit_transform(X_train)
X_val_enc   = preprocess.transform(X_valid)
X_test_enc  = preprocess.transform(X_test)

print(X_train_enc.shape)
print(X_val_enc.shape)
print(X_test_enc.shape)

# 1) map best params from tuner to lgb.train
bp = best_params_lgb
g  = lambda k, d=None: bp.get(f"model__estimator__{k}", d)

lgb_params = {
    "seed": 42,                       # master seed (alias: random_state)
    "bagging_seed": 42,
    "feature_fraction_seed": 42,
    "data_random_seed": 42,
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": g("learning_rate", 0.05),
    "num_leaves": g("num_leaves", 127),
    "max_depth": g("max_depth", -1),
    "feature_fraction": g("colsample_bytree", 0.8),   # alias
    "bagging_fraction": g("subsample", 0.8),          # alias
    "bagging_freq": bp.get("model__estimator__subsample_freq", 0),
    "min_data_in_leaf": g("min_child_samples", 20),
    "lambda_l1": g("reg_alpha", 0.0),
    "lambda_l2": g("reg_lambda", 0.1),
    "verbosity": -1,
    "num_threads": -1,
    # Optional GPU:
    "device_type": "gpu",
    # Optional speed knob during refit:
    "max_bin": 255,
}
num_boost_round = g("n_estimators", 4000)
early_stopping_rounds = 100

# 4) Train per-horizon on TRAIN, early-stop on VALID; predict both VALID and TEST
models_lgb = []

for h in range(HORIZON):  # 0..23 => t+1..t+24
    Xtr_h = np.hstack([X_train_enc, Z_train[:, h, :]]).astype(DTYPE)
    Xva_h = np.hstack([X_val_enc,   Z_val[:,   h, :]]).astype(DTYPE)
    Xte_h = np.hstack([X_test_enc,  Z_test[:,  h, :]]).astype(DTYPE)

    dtrain = lgb.Dataset(Xtr_h, label=Y_train[:, h])
    dvalid = lgb.Dataset(Xva_h, label=Y_valid[:, h], reference=dtrain)

    booster = lgb.train(
        params=lgb_params,
        train_set=dtrain,
        num_boost_round=num_boost_round,
        valid_sets=[dvalid],
        callbacks=[lgb.early_stopping(early_stopping_rounds), lgb.log_evaluation(-1)],
    )
    models_lgb.append((booster, Xte_h))

# Predict per-horizon (use each horizon’s Xte_h)
Y_hat_lgb_z = np.column_stack([
    m.predict(Xh, num_iteration=m.best_iteration) for (m, Xh) in models_lgb
])

# 7) Metrics (same as yours)
def safe_mape(y_true, y_pred, eps=1e-6):
    y_true = np.asarray(y_true)
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), eps, None))) * 100.0

mae_h_lgb_z = np.mean(np.abs(Y_test - Y_hat_lgb_z), axis=0)
rmse_h_lgb_z = [math.sqrt(mean_squared_error(Y_test[:, h], Y_hat_lgb_z[:, h])) for h in range(HORIZON)]
nrmse_h_lgb_z = [rm / (np.mean(Y_test[:, h]) + 1e-6) * 100 for h, rm in enumerate(rmse_h_lgb_z)]
mape_h_lgb_z  = [safe_mape(Y_test[:, h], Y_hat_lgb_z[:, h]) for h in range(HORIZON)]

print("Per-horizon RMSE:", [round(x,2) for x in rmse_h_lgb_z])
print("Avg RMSE:  {:.3f}".format(np.mean(rmse_h_lgb_z)))
print("Avg nRMSE: {:.2f}%".format(np.mean(nrmse_h_lgb_z)))
print("Avg MAE:  {:.3f}".format(np.mean(mae_h_lgb_z)))
print("Avg MAPE:  {:.2f}%".format(np.mean(mape_h_lgb_z)))


(44689, 155)
(1344, 155)
(1344, 155)
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[299]	valid_0's rmse: 220.981
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[232]	valid_0's rmse: 214.229
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[300]	valid_0's rmse: 205.801
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[300]	valid_0's rmse: 208.569
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[299]	valid_0's rmse: 203.6
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[298]	valid_0's rmse: 207.597
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[298]	valid_0's

# Results

## 70-15-15 split approach

In [ ]:
print("XGBoost Results:")
print("Per-h RMSE:", [round(x,2) for x in rmse_h_xgb])
print("Avg RMSE:  {:.3f}".format(np.mean(rmse_h_xgb)))
print("Avg nRMSE: {:.2f}%".format(np.mean(nrmse_h_xgb)))
print("Avg MAPE:  {:.2f}%".format(np.mean(mape_h_xgb)))

XGBoost Results:
Per-h RMSE: [296.59, 325.64, 270.0, 270.46, 277.15, 265.78, 267.23, 259.59, 263.36, 274.65, 272.44, 275.65, 289.8, 283.15, 271.81, 291.29, 274.61, 271.92, 265.46, 284.73, 268.49, 266.76, 286.56, 301.62]
Avg RMSE:  278.114
Avg nRMSE: 11.85%
Avg MAPE:  9.00%


In [ ]:
print("LightGB results:")
print("Per-horizon RMSE:", [round(x,2) for x in rmse_h_lgb])
print("Avg RMSE:  {:.3f}".format(np.mean(rmse_h_lgb)))
print("Avg nRMSE: {:.2f}%".format(np.mean(nrmse_h_lgb)))
print("Avg MAPE:  {:.2f}%".format(np.mean(mape_h_lgb)))

LightGB results:
Per-horizon RMSE: [274.6, 285.04, 273.35, 276.21, 281.96, 273.42, 270.18, 261.52, 271.67, 277.64, 276.35, 277.89, 280.05, 284.98, 276.67, 274.92, 275.68, 273.08, 265.77, 281.67, 272.1, 267.45, 286.04, 295.74]
Avg RMSE:  276.415
Avg nRMSE: 11.78%
Avg MAPE:  8.74%


In [ ]:
Y_hat_xgb.shape, Y_hat_lgb.shape, Y_test.shape

((7107, 24), (7107, 24), (7107, 24))

In [ ]:
for h in range(HORIZON):
    print(f"xgb_t+{h+1}:", Y_hat_xgb[:, h][0])
    print(f"lgb_t+{h+1}:", Y_hat_lgb[:, h][0])
    print(f"y_true_t+{h+1}:", Y_test[:, h][0])

xgb_t+1: 1988.4346
lgb_t+1: 2005.6319082389916
y_true_t+1: 1866.44
xgb_t+2: 1563.6069
lgb_t+2: 1556.2699649141357
y_true_t+2: 1549.87
xgb_t+3: 1424.0342
lgb_t+3: 1399.2779444772386
y_true_t+3: 1372.76
xgb_t+4: 1391.7781
lgb_t+4: 1400.8607798082196
y_true_t+4: 1304.0
xgb_t+5: 1439.0226
lgb_t+5: 1467.0696642781
y_true_t+5: 1304.0
xgb_t+6: 1655.1913
lgb_t+6: 1595.92168264926
y_true_t+6: 1516.86
xgb_t+7: 1915.626
lgb_t+7: 1966.0344576286554
y_true_t+7: 2003.4
xgb_t+8: 2836.6973
lgb_t+8: 2878.060269996576
y_true_t+8: 2938.18
xgb_t+9: 3604.7205
lgb_t+9: 3593.8982417774014
y_true_t+9: 4034.65
xgb_t+10: 4490.1543
lgb_t+10: 4495.7502563464195
y_true_t+10: 4437.05
xgb_t+11: 4637.315
lgb_t+11: 4612.101847856309
y_true_t+11: 4694.4
xgb_t+12: 4743.7603
lgb_t+12: 4703.490315495516
y_true_t+12: 4694.4
xgb_t+13: 4709.4863
lgb_t+13: 4699.345637924606
y_true_t+13: 4694.4
xgb_t+14: 4733.4927
lgb_t+14: 4706.422566885368
y_true_t+14: 4694.4
xgb_t+15: 4724.5723
lgb_t+15: 4704.18478239122
y_true_t+15: 4694.4

In [ ]:
Y_hat_ens = 0.5 * (Y_hat_xgb + Y_hat_lgb)
print("Ensemble Avg RMSE:", np.mean([math.sqrt(mean_squared_error(Y_test[:,h], Y_hat_ens[:,h])) for h in range(24)]))

Ensemble Avg RMSE: 273.89086614940715


## Results (8 weeks test set split approach)

### without future known ahead Z

XGBoost:
- Per-h RMSE: [286.11, 295.73, 272.58, 280.61, 275.54, 271.19, 273.26, 263.52, 273.82, 286.92, 283.05, 296.13, 292.93, 312.29, 307.13, 304.76, 299.94, 282.78, 287.76, 306.48, 300.22, 301.2, 319.0, 304.15]
- Avg RMSE:  290.712
- Avg nRMSE: 11.75%
- Avg MAPE:  8.41%

LightGBM:
- Per-horizon RMSE: [291.95, 303.93, 272.19, 277.76, 274.24, 268.7, 274.16, 265.82, 268.67, 284.44, 287.53, 300.76, 290.27, 322.4, 303.28, 301.28, 319.59, 282.12, 289.37, 332.45, 305.16, 299.9, 329.33, 306.77]
- Avg RMSE:  293.836
- Avg nRMSE: 11.87%
- Avg MAPE:  8.33%

### with  future known ahead Z

XGBoost:
- Per-h RMSE: [284.64, 289.16, 264.31, 271.62, 269.46, 274.13, 268.79, 263.21, 269.48, 286.15, 293.18, 295.98, 269.63, 302.86, 284.66, 295.39, 295.11, 276.92, 280.03, 304.86, 296.26, 292.53, 318.07, 300.43]
- Avg RMSE:  285.286
- Avg nRMSE: 11.53%
- Avg MAPE:  8.13%

LightGBM:
- Per-horizon RMSE: [289.67, 291.72, 264.62, 270.11, 268.9, 268.16, 269.25, 260.04, 267.4, 278.51, 279.64, 277.78, 271.73, 294.02, 286.94, 294.02, 294.33, 274.3, 278.14, 308.48, 296.83, 288.75, 320.95, 305.96]
- Avg RMSE:  283.343
- Avg nRMSE: 11.45%
- Avg MAPE:  7.81%